## Notebook 1: Database Setup for Solana Alpha Bot
### This notebook sets up the database schema and models

In [ ]:
# Import dependencies
import sqlite3
import asyncio
import json
from datetime import datetime, timezone
from typing import List, Dict, Optional
import pandas as pd
import os
from dotenv import load_dotenv

In [2]:
# Load environment variables
load_dotenv()

print("🗄️ Setting up Solana Alpha Bot Database...")
print(f"Database URL: {os.getenv('DATABASE_URL', 'sqlite:///./solana_alpha.db')}")

🗄️ Setting up Solana Alpha Bot Database...
Database URL: sqlite:///./solana_alpha.db


In [3]:
# Database connection setup
class DatabaseManager:
    def __init__(self, db_path: str = "./solana_alpha.db"):
        self.db_path = db_path
        self.connection = None
    
    def connect(self):
        """Create database connection"""
        self.connection = sqlite3.connect(self.db_path)
        self.connection.row_factory = sqlite3.Row  # Enable column access by name
        return self.connection
    
    def execute_query(self, query: str, params: tuple = ()):
        """Execute a query and return results"""
        cursor = self.connection.cursor()
        cursor.execute(query, params)
        return cursor.fetchall()
    
    def execute_update(self, query: str, params: tuple = ()):
        """Execute an update/insert query"""
        cursor = self.connection.cursor()
        cursor.execute(query, params)
        self.connection.commit()
        return cursor.rowcount
    
    def close(self):
        """Close database connection"""
        if self.connection:
            self.connection.close()

In [4]:

# Test database connection
db = DatabaseManager()
db.connect()
print("✅ Database connection established")

✅ Database connection established


In [ ]:
# Create database tables
def create_tables(db: DatabaseManager):
    """Create all necessary tables for the alpha bot"""
    
    # Users table - stores Telegram users and their preferences
    users_table = """
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        chat_id INTEGER UNIQUE NOT NULL,
        username TEXT,
        first_name TEXT,
        subscription_level TEXT DEFAULT 'moderate',
        min_alpha_score INTEGER DEFAULT 50,
        max_alerts_per_hour INTEGER DEFAULT 5,
        is_premium BOOLEAN DEFAULT FALSE,
        is_active BOOLEAN DEFAULT TRUE,
        quiet_hours_start TEXT DEFAULT '23:00',
        quiet_hours_end TEXT DEFAULT '07:00',
        timezone TEXT DEFAULT 'UTC',
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    
    # Wallets table - stores tracked wallet addresses and their performance
    wallets_table = """
    CREATE TABLE IF NOT EXISTS wallets (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        address TEXT UNIQUE NOT NULL,
        label TEXT,
        total_trades INTEGER DEFAULT 0,
        profitable_trades INTEGER DEFAULT 0,
        total_profit REAL DEFAULT 0.0,
        win_rate REAL DEFAULT 0.0,
        avg_hold_time_hours INTEGER DEFAULT 0,
        last_activity TIMESTAMP,
        is_verified BOOLEAN DEFAULT FALSE,
        discovery_method TEXT,
        performance_score REAL DEFAULT 0.0,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    
    # Tokens table - stores token information and metadata  
    tokens_table = """
    CREATE TABLE IF NOT EXISTS tokens (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        address TEXT UNIQUE NOT NULL,
        symbol TEXT,
        name TEXT,
        decimals INTEGER DEFAULT 9,
        market_cap REAL,
        volume_24h REAL,
        price_usd REAL,
        liquidity REAL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    
    # Alpha alerts table - stores generated alerts and their outcomes
    alpha_alerts_table = """
    CREATE TABLE IF NOT EXISTS alpha_alerts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        token_address TEXT NOT NULL,
        alpha_score REAL NOT NULL,
        alert_level TEXT NOT NULL,
        buying_wallets_count INTEGER NOT NULL,
        total_volume REAL,
        alert_data JSON,
        sent_to_users INTEGER DEFAULT 0,
        price_at_alert REAL,
        price_after_1h REAL,
        price_after_24h REAL,
        outcome_profitable BOOLEAN,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (token_address) REFERENCES tokens (address)
    )
    """
    
    # Wallet transactions table - stores individual wallet transactions
    wallet_transactions_table = """
    CREATE TABLE IF NOT EXISTS wallet_transactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        wallet_address TEXT NOT NULL,
        token_address TEXT NOT NULL,
        transaction_hash TEXT UNIQUE NOT NULL,
        transaction_type TEXT NOT NULL, -- 'buy' or 'sell'
        amount REAL NOT NULL,
        price_usd REAL,
        value_usd REAL,
        block_number INTEGER,
        transaction_time TIMESTAMP NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (wallet_address) REFERENCES wallets (address),
        FOREIGN KEY (token_address) REFERENCES tokens (address)
    )
    """
    
    # User wallet watchlists - users can track specific wallets
    user_watchlists_table = """
    CREATE TABLE IF NOT EXISTS user_watchlists (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_chat_id INTEGER NOT NULL,
        wallet_address TEXT NOT NULL,
        watchlist_name TEXT DEFAULT 'default',
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_chat_id) REFERENCES users (chat_id),
        FOREIGN KEY (wallet_address) REFERENCES wallets (address)
    )
    """
    # Execute table creation
    tables = [
        ("users", users_table),
        ("wallets", wallets_table), 
        ("tokens", tokens_table),
        ("alpha_alerts", alpha_alerts_table),
        ("wallet_transactions", wallet_transactions_table),
        ("user_watchlists", user_watchlists_table)
    ]
    
    for table_name, query in tables:
        try:
            db.execute_update(query)
            print(f"✅ Created table: {table_name}")
        except Exception as e:
            print(f"❌ Error creating {table_name}: {e}")

In [ ]:
# Create all tables
create_tables(db)

✅ Created table: users
✅ Created table: wallets
✅ Created table: tokens
✅ Created table: alpha_alerts
✅ Created table: wallet_transactions
✅ Created table: user_watchlists


In [8]:
# Database helper functions
class DatabaseHelpers:
    def __init__(self, db: DatabaseManager):
        self.db = db
    
    def add_user(self, chat_id: int, username: str = None, first_name: str = None, subscription_level: str = "moderate"):
        """Add new Telegram user"""
        query = """
        INSERT OR REPLACE INTO users (chat_id, username, first_name, subscription_level)
        VALUES (?, ?, ?, ?)
        """
        self.db.execute_update(query, (chat_id, username, first_name, subscription_level))
        print(f"✅ Added user: {chat_id}")
    
    def get_user(self, chat_id: int) -> Dict:
        """Get user information"""
        query = "SELECT * FROM users WHERE chat_id = ?"
        result = self.db.execute_query(query, (chat_id,))
        return dict(result[0]) if result else None
    
    def add_wallet(self, address: str, label: str = None, discovery_method: str = "manual"):
        """Add wallet to tracking database"""
        query = """
        INSERT OR IGNORE INTO wallets (address, label, discovery_method)
        VALUES (?, ?, ?)
        """
        self.db.execute_update(query, (address, label, discovery_method))
        print(f"✅ Added wallet: {address[:8]}...{address[-6:]}")
    
    def get_tracked_wallets(self, limit: int = 100) -> List[Dict]:
        """Get all tracked wallets"""
        query = """
        SELECT address, label, win_rate, total_profit, performance_score
        FROM wallets 
        WHERE is_verified = TRUE
        ORDER BY performance_score DESC
        LIMIT ?
        """
        results = self.db.execute_query(query, (limit,))
        return [dict(row) for row in results]
    
    def add_token(self, address: str, symbol: str = None, name: str = None):
        """Add token to database"""
        query = """
        INSERT OR IGNORE INTO tokens (address, symbol, name)
        VALUES (?, ?, ?)
        """
        self.db.execute_update(query, (address, symbol, name))
    
    def record_alpha_alert(self, token_address: str, alpha_score: float, alert_level: str, 
                          buying_wallets_count: int, alert_data: Dict):
        """Record generated alpha alert"""
        query = """
        INSERT INTO alpha_alerts 
        (token_address, alpha_score, alert_level, buying_wallets_count, alert_data)
        VALUES (?, ?, ?, ?, ?)
        """
        self.db.execute_update(query, (
            token_address, alpha_score, alert_level, 
            buying_wallets_count, json.dumps(alert_data)
        ))
        print(f"🚨 Recorded alpha alert: {alpha_score}/100 for {token_address[:8]}...")
    
    def get_user_subscribers(self, min_score: float = 50) -> List[int]:
        """Get list of chat_ids for users who want alerts above min_score"""
        query = """
        SELECT chat_id FROM users 
        WHERE is_active = TRUE AND min_alpha_score <= ?
        """
        results = self.db.execute_query(query, (min_score,))
        return [row[0] for row in results]

In [ ]:
# Initialize database helpers
db_helpers = DatabaseHelpers(db)
# Test database connection
db.connect()
print("✅ Database helpers initialized")


✅ Database helpers initialized


In [11]:
# Test database with sample data
print("\n🧪 Testing database with sample data...")

# Add sample users
sample_users = [
    (123456789, "degen_trader", "John", "high"),
    (987654321, "alpha_hunter", "Sarah", "critical"),
    (555666777, "moon_farmer", "Mike", "moderate")
]

for chat_id, username, first_name, level in sample_users:
    db_helpers.add_user(chat_id, username, first_name, level)

# Add sample wallets (these would be discovered wallets in production)
sample_wallets = [
    ("7xKXtg2CW87d97TXJSDpbD5jBkheTqA83TZRuJosgAsU", "Top Sniper #1", "algorithm"),
    ("GDhDU8aLbLN4wj2kqGJgwjQzK8P3F5v7V8X2a1b2c3d4", "Whale Tracker", "social"),
    ("5FHwkrdxnt4P28a62GQ8tGJf8f7e9g5h4i3j2k1l9m8n", "MEV Bot", "onchain_analysis"),
]

for address, label, method in sample_wallets:
    db_helpers.add_wallet(address, label, method)

# Add sample tokens
sample_tokens = [
    ("EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v", "USDC", "USD Coin"),
    ("So11111111111111111111111111111111111111112", "SOL", "Solana"),
    ("DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7YaB1pPB263", "BONK", "Bonk"),
]

for address, symbol, name in sample_tokens:
    db_helpers.add_token(address, symbol, name)

# Test alpha alert recording
sample_alert = {
    "token_symbol": "NEWMEME",
    "buying_wallets": ["7xKX...", "GDhD...", "5FHw..."],
    "detection_time": datetime.now(timezone.utc).isoformat()
}

db_helpers.record_alpha_alert(
    token_address="A1B2C3D4E5F6G7H8I9J0K1L2M3N4O5P6Q7R8S9T0U1V2",
    alpha_score=85.5,
    alert_level="high",
    buying_wallets_count=3,
    alert_data=sample_alert
)


🧪 Testing database with sample data...
✅ Added user: 123456789
✅ Added user: 987654321
✅ Added user: 555666777
✅ Added wallet: 7xKXtg2C...osgAsU
✅ Added wallet: GDhDU8aL...b2c3d4
✅ Added wallet: 5FHwkrdx...1l9m8n
🚨 Recorded alpha alert: 85.5/100 for A1B2C3D4...


In [12]:
# Database statistics and validation
print("\n📊 Database Statistics:")

# Check table row counts
tables = ["users", "wallets", "tokens", "alpha_alerts", "wallet_transactions", "user_watchlists"]
for table in tables:
    count_query = f"SELECT COUNT(*) FROM {table}"
    count = db.execute_query(count_query)[0][0]
    print(f"  {table}: {count} records")

# Test user retrieval
print("\n👥 Sample Users:")
users = db.execute_query("SELECT chat_id, username, subscription_level FROM users")
for user in users:
    print(f"  {user[1]} (ID: {user[0]}) - Level: {user[2]}")

# Test wallet data
print("\n💼 Sample Wallets:")
wallets = db.execute_query("SELECT address, label, discovery_method FROM wallets LIMIT 5")
for wallet in wallets:
    addr_short = f"{wallet[0][:8]}...{wallet[0][-6:]}"
    print(f"  {addr_short} - {wallet[1]} (via {wallet[2]})")

# Test subscriber list
subscribers = db_helpers.get_user_subscribers(min_score=70)
print(f"\n📢 Subscribers for high-score alerts (70+): {len(subscribers)} users")


📊 Database Statistics:
  users: 3 records
  wallets: 3 records
  tokens: 3 records
  alpha_alerts: 1 records
  wallet_transactions: 0 records
  user_watchlists: 0 records

👥 Sample Users:
  degen_trader (ID: 123456789) - Level: high
  alpha_hunter (ID: 987654321) - Level: critical
  moon_farmer (ID: 555666777) - Level: moderate

💼 Sample Wallets:
  7xKXtg2C...osgAsU - Top Sniper #1 (via algorithm)
  GDhDU8aL...b2c3d4 - Whale Tracker (via social)
  5FHwkrdx...1l9m8n - MEV Bot (via onchain_analysis)

📢 Subscribers for high-score alerts (70+): 3 users


In [13]:

# Database schema validation
def validate_database_schema():
    """Validate that all tables and columns exist"""
    
    expected_tables = {
        'users': ['chat_id', 'username', 'subscription_level', 'min_alpha_score'],
        'wallets': ['address', 'label', 'win_rate', 'total_profit'],
        'tokens': ['address', 'symbol', 'name'],
        'alpha_alerts': ['token_address', 'alpha_score', 'alert_level'],
        'wallet_transactions': ['wallet_address', 'token_address', 'transaction_type'],
        'user_watchlists': ['user_chat_id', 'wallet_address']
    }
    
    print("🔍 Validating database schema...")
    
    for table_name, required_columns in expected_tables.items():
        # Check if table exists
        table_check = db.execute_query(
            "SELECT name FROM sqlite_master WHERE type='table' AND name=?", 
            (table_name,)
        )
        
        if not table_check:
            print(f"❌ Missing table: {table_name}")
            continue
            
        # Check columns
        columns_query = f"PRAGMA table_info({table_name})"
        columns = db.execute_query(columns_query)
        column_names = [col[1] for col in columns]
        
        missing_columns = [col for col in required_columns if col not in column_names]
        if missing_columns:
            print(f"❌ {table_name}: Missing columns {missing_columns}")
        else:
            print(f"✅ {table_name}: All required columns present")

In [14]:
# Run validation
validate_database_schema()

🔍 Validating database schema...
✅ users: All required columns present
✅ wallets: All required columns present
✅ tokens: All required columns present
✅ alpha_alerts: All required columns present
✅ wallet_transactions: All required columns present
✅ user_watchlists: All required columns present


In [15]:

# Export sample data for testing
print("\n💾 Exporting sample data...")

# Create sample data files for testing other notebooks
sample_data = {
    "successful_wallets": [
        {
            "address": "7xKXtg2CW87d97TXJSDpbD5jBkheTqA83TZRuJosgAsU",
            "win_rate": 0.85,
            "total_profit": 125000,
            "avg_trade_size": 5000,
            "specialties": ["low_cap_gems", "meme_coins"]
        },
        {
            "address": "GDhDU8aLbLN4wj2kqGJgwjQzK8P3F5v7V8X2a1b2c3d4", 
            "win_rate": 0.72,
            "total_profit": 89000,
            "avg_trade_size": 3500,
            "specialties": ["defi_tokens", "gaming"]
        }
    ],
    "test_tokens": [
        {
            "address": "A1B2C3D4E5F6G7H8I9J0K1L2M3N4O5P6Q7R8S9T0U1V2",
            "symbol": "ALPHATEST",
            "name": "Alpha Test Token",
            "market_cap": 1500000,
            "volume_24h": 250000
        }
    ]
}

# Ensure data directory exists
os.makedirs("data", exist_ok=True)

# Export to JSON files
with open("data/sample_wallets.json", "w") as f:
    json.dump(sample_data["successful_wallets"], f, indent=2)

with open("data/sample_tokens.json", "w") as f:
    json.dump(sample_data["test_tokens"], f, indent=2)

print("✅ Sample data exported to data/ directory")


💾 Exporting sample data...
✅ Sample data exported to data/ directory


In [16]:

# Cleanup and next steps
print("\n🎯 Database setup completed successfully!")
print("\nNext steps:")
print("1. ✅ Database schema created")
print("2. ✅ Sample data inserted") 
print("3. ✅ Helper functions tested")
print("4. ✅ Data exported for other notebooks")
print("\n➡️  Ready to move to: 02_solana_client.ipynb")

# Close database connection
db.close()
print("🔒 Database connection closed")


🎯 Database setup completed successfully!

Next steps:
1. ✅ Database schema created
2. ✅ Sample data inserted
3. ✅ Helper functions tested
4. ✅ Data exported for other notebooks

➡️  Ready to move to: 02_solana_client.ipynb
🔒 Database connection closed


In [1]:
import requests

url = "https://pro-api.coingecko.com/api/v3/onchain/networks/solana/new_pools"

headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": "CG-nm5ynnbGBWTugQhVY2vNMWbP"
}

response = requests.get(url, headers=headers)

print(response.text)

{"data":[{"id":"solana_7k8Y9y5gFETgSr5dsUa89kwtiT2HmB62BgqukAEyS8gC","type":"pool","attributes":{"base_token_price_usd":"0.000067235367725324682938396364448130119907651781025477918135166085","base_token_price_native_currency":"0.000000322227167687477","quote_token_price_usd":"208.658283557689000827994196967888618037162998174","quote_token_price_native_currency":"1.0","base_token_price_quote_token":"0.0000003222271677","quote_token_price_base_token":"3103400.64488257","address":"7k8Y9y5gFETgSr5dsUa89kwtiT2HmB62BgqukAEyS8gC","name":"VANILLA / SOL","pool_created_at":"2025-09-08T08:54:13Z","fdv_usd":null,"market_cap_usd":null,"price_change_percentage":{"m5":"8.305","m15":"8.305","m30":"8.305","h1":"8.305","h6":"8.305","h24":"8.305"},"transactions":{"m5":{"buys":21,"sells":0,"buyers":21,"sellers":0},"m15":{"buys":21,"sells":0,"buyers":21,"sellers":0},"m30":{"buys":21,"sells":0,"buyers":21,"sellers":0},"h1":{"buys":21,"sells":0,"buyers":21,"sellers":0},"h6":{"buys":21,"sells":0,"buyers":21,"

In [2]:
import pandas as pd
r_pd = pd.json_normalize(response.json()['data'])

In [3]:
r_pd

,id,type,attributes.base_token_price_usd,attributes.base_token_price_native_currency,attributes.quote_token_price_usd,attributes.quote_token_price_native_currency,attributes.base_token_price_quote_token,attributes.quote_token_price_base_token,attributes.address,attributes.name,...,attributes.volume_usd.h1,attributes.volume_usd.h6,attributes.volume_usd.h24,attributes.reserve_in_usd,relationships.base_token.data.id,relationships.base_token.data.type,relationships.quote_token.data.id,relationships.quote_token.data.type,relationships.dex.data.id,relationships.dex.data.type
0,solana_7k8Y9y5gFETgSr5dsUa89kwtiT2HmB62BgqukAE...,pool,0.00006723536772532468293839636444813011990765...,0.000000322227167687477,208.658283557689000827994196967888618037162998174,1.0,0.0000003222271677,3103400.64488257,7k8Y9y5gFETgSr5dsUa89kwtiT2HmB62BgqukAEyS8gC,VANILLA / SOL,...,829.623339775,829.623339775,829.623339775,36567.0512,solana_2cH6wfT4jBbfiNo5QUenze3s59jrSGQc1hLpZav...,token,solana_So1111111111111111111111111111111111111...,token,pumpswap,dex
1,solana_HJz3RqttNYNEHhkke5t6sCLhtLrHWqUWZRVAeFg...,pool,0.00006410552607083137142184206449297080917757...,0.000000304153361396689,208.657090398718424695568697024333634682139167979,1.0,0.0000003041533614,3287815.05293233,HJz3RqttNYNEHhkke5t6sCLhtLrHWqUWZRVAeFgLtuhk,OLM / SOL,...,610.49371681,610.49371681,610.49371681,35852.7722,solana_27z5da3LXeCBRVQHno7MrisMncxzDX4GShT1kHn...,token,solana_So1111111111111111111111111111111111111...,token,pumpswap,dex
2,solana_7GhimLGs5GwKV7imNzNYUWwqKaavdv478MAb4rZ...,pool,0.00000835884332492660607321504122208348425927...,0.0000000400599639870784,208.658283557689000827994196967888618037162998174,1.0,0.00000004005996399,24962578.6064749,7GhimLGs5GwKV7imNzNYUWwqKaavdv478MAb4rZcyzbr,THUG / SOL,...,1892.7701791406,1892.7701791406,1892.7701791406,5813.341,solana_76GGxo5tNadEE1tYHbRzcJCaUkBGB1fLFpWVXKg...,token,solana_So1111111111111111111111111111111111111...,token,pump-fun,dex
3,solana_HikNoVKPb1Aasj6j86PEbUv7fyqUy5rmkdJBtJc...,pool,0.00000705548999822081437334468297118883964400...,0.0000000338132399569324,208.660572225770864718533103143642428733682396148,1.0,0.00000003381323996,29574214.1620765,HikNoVKPb1Aasj6j86PEbUv7fyqUy5rmkdJBtJcAuLLX,67stream / SOL,...,1968.1478634279,1968.1478634279,1968.1478634279,5632.1416,solana_3KYzGXmmt7Wy9GWLKQsLrsh7mF4u5jKMKeeVYdv...,token,solana_So1111111111111111111111111111111111111...,token,pump-fun,dex
4,solana_BjP7UsPLjGzS1h7ggKL8bwabzE5x4kAc5UMkKmA...,pool,0.00001921622154352456545873615434511058605176...,0.0000000920947450518202,208.657090398718424695568697024333634682139167979,1.0,0.00000009209474505,10858382.8473309,BjP7UsPLjGzS1h7ggKL8bwabzE5x4kAc5UMkKmAYoVSG,NASDAQ / SOL,...,1084.9438070765,1084.9438070765,1084.9438070765,36858.6929,solana_GNNsMXrvhpFZ8HVWfFFsKw7GhrNg8rFmDMVZTGM...,token,solana_So1111111111111111111111111111111111111...,token,pumpswap,dex
5,solana_AZZNJzYSw7TTPXMzyrgziiEHpmphfCTMuqLRegV...,pool,0.00007181459705452598934268219266721762923697...,0.000000348096976629121,208.657090398718424695568697024333634682139167979,1.0,0.0000003480969766,2872762.67,AZZNJzYSw7TTPXMzyrgziiEHpmphfCTMuqLRegVz1dSU,Trench / SOL,...,718.3360893121,718.3360893121,718.3360893121,28332.6024,solana_4jwJBF9EE8pbRkH87cQvUjZCXasxWcUmG3VwEn4...,token,solana_So1111111111111111111111111111111111111...,token,meteora-damm-v2,dex
6,solana_8hoeTtDS3cNRxHAkJcyovL51X36Mhf1UMVLFfmo...,pool,0.00000585468660209715755614333669532077708547...,0.0000000280568372506804,208.67236566214101574369265622121844967076426596,1.0,0.00000002805683725,35641936.083717,8hoeTtDS3cNRxHAkJcyovL51X36Mhf1UMVLFfmonSaWm,MERL / SOL,...,1318.4039690942,1318.4039690942,1318.4039690942,4643.0224,solana_3DNVG8W39x8JRj24bzkeJoeYgwQN5RFHSB1VueR...,token,solana_So1111111111111111111111111111111111111...,token,pump-fun,dex
7,solana_4hoDiM6MU8PdG35b1aGM2WgR1w1mthfNwNFBWSg...,pool,0.00000584559419136524812598712000039725012122...,0.000000028033247339496,208.656586116575404549080734390561171733310